In [1]:
data_path = "/kaggle/input/smai-s-25-section-a-project-phase-2"
import os
import pandas as pd
import numpy as np
images_train_path = os.path.join(data_path, 'images_train/images_train')
images_val_path = os.path.join(data_path, 'images_val/images_val')
labels_train = os.path.join(data_path, 'labels_train.csv')
labels_val = os.path.join(data_path, 'labels_val.csv')

import pandas as pd
import numpy as np
df = pd.read_csv(labels_train)
# df['timestamp'] = pd.to_datetime(df['timestamp'], format='%H:%M').dt.time

print(df.head())


       filename timestamp  latitude  longitude  angle  Region_ID
0  img_0000.jpg     15:03    219698     144782    133          2
1  img_0001.jpg     15:05    219844     144621    312          2
2  img_0002.jpg     15:05    219844     144621    359          2
3  img_0003.jpg     17:11    219514     145016    131          2
4  img_0004.jpg     17:00    220182     144211     45          2


In [2]:

import cv2
import os
image = cv2.imread(os.path.join(images_train_path, df['filename'][0]))
print(image.shape)

# in the df change all the file names to full path 
df['filename'] = df['filename'].apply(lambda x: os.path.join(images_train_path, x))
df.head()

# some data cleaning, if angle > 360 remove
df = df[df['angle'] < 361]
df.head()
df['angle_rad'] = np.deg2rad(df['angle'])
df['angle_sin'] = np.sin(df['angle_rad'])
df['angle_cos'] = np.cos(df['angle_rad'])



(256, 256, 3)


In [3]:
validation_df = pd.read_csv(labels_val)
validation_df['filename'] = validation_df['filename'].apply(lambda x: os.path.join(images_val_path, x))
validation_df.head()

validation_df['angle_rad'] = np.deg2rad(validation_df['angle'])
validation_df['angle_sin'] = np.sin(validation_df['angle_rad'])
validation_df['angle_cos'] = np.cos(validation_df['angle_rad'])


In [4]:
from torchvision import transforms

train_transforms = transforms.Compose([
    # transforms.RandomResizedCrop(256, scale=(0.8,1.0)),
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
])
# can add a randomposterize 
# random adjust sharpness 
# random autocontrast 

train_transforms_2 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomAutocontrast(),
    transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.RandomPosterize(bits=4),
    transforms.ToTensor()
])

In [5]:
from torch.utils.data import Dataset
from PIL import Image


In [6]:
class MyDataset(Dataset):
    def __init__(self, df, transform1=None, transform2 = None, validation=False):
        self.df = df
        self.transform1 = transform1
        self.transform2 = transform2 
        self.triple_len = len(df) * 3
        self.validation = validation
    

    def __len__(self):
        if self.validation:
            return len(self.df)
        else:
            return self.triple_len

    def __getitem__(self, idx):
        row = self.df.iloc[idx % len(self.df)]
        img = Image.open(f"{row.filename}").convert('RGB')
        img = img.resize((224, 224))

        if self.validation or idx % len(self.df) == 0:
            transformed_img = transforms.ToTensor()(img)
        elif idx > len(self.df) and idx < 2 * len(self.df):
            transformed_img = self.transform1(img) if self.transform1 else transforms.ToTensor()(img)
        else:
            transformed_img = self.transform2(img) if self.transform2 else transforms.ToTensor()(img)

        label = row['angle']
        return transformed_img, label


train_dataset = MyDataset(df, transform1=train_transforms, transform2 = train_transforms_2) 
print(len(train_dataset))
# img_sample, label = train_dataset[len(df) + 4]
# print(img_sample.shape)
# print(label)
validation_dataset = MyDataset(validation_df, validation=True)

print(len(validation_dataset))
# img_sample, label = validation_dataset[0]
# print(img_sample.shape)
# print(label)



19620
369


In [7]:
import torch
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=32, shuffle=False)
# device = 
print(len(train_loader))
print(len(validation_loader))
# print(device)
print(train_loader.dataset[0][0].shape)
# sample_image, sample_label = train_loader.dataset[0]

614
12
torch.Size([3, 224, 224])


In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import os
from tqdm import tqdm


In [9]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb-key")

wandb.login(key = secret_value_0)

wandb.init(
    project = "smai-project-angleID",
    name = 'method2-efficientnetb2-angle-regression'
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: v4ishnavi (v4ishnavi-iiit-hyderabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ['CUDA_LAUNCH_BLOCKING'] = "0"

model_eb2 = models.efficientnet_b2(pretrained=True)
in_features = model_eb2.classifier[1].in_features
model_eb2.classifier[1] = nn.Linear(in_features, 2)  
model_eb2 = model_eb2.to(device)

total_blocks = len(model_eb2.features)
freeze_blocks = int(total_blocks * 0.7)

for i, block in enumerate(model_eb2.features):
    if i < freeze_blocks:
        for param in block.parameters():
            param.requires_grad = False
    else:
        for param in block.parameters():
            param.requires_grad = True

for param in model_eb2.classifier.parameters():
    param.requires_grad = True

optimizer = optim.Adam([
    {'params': model_eb2.features[freeze_blocks:].parameters(), 'lr': 0.0005},
    {'params': model_eb2.classifier.parameters(), 'lr': 0.001}
])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

criterion = nn.MSELoss()

num_epochs = 40
best_val_loss = float('inf')
patience = 5
patience_counter = 0
best_model_state = None



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B2_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 166MB/s] 
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_l

In [11]:
# Helper functions
def angle_to_sincos(angles):
    """Convert angles in degrees to sine and cosine components"""
    angles = angles.float()  # Make sure we're working with float tensors for math operations
    angles_rad = torch.deg2rad(angles)
    sin_values = torch.sin(angles_rad)
    cos_values = torch.cos(angles_rad)
    return torch.cat([sin_values, cos_values], dim=1)

def sincos_to_angle(sin_cos):
    """Convert sine and cosine components back to angles in degrees (as integers)"""
    sin, cos = sin_cos[:, 0], sin_cos[:, 1]
    angles_rad = torch.atan2(sin, cos)
    angles_deg = torch.rad2deg(angles_rad)
    angles_int = torch.round((angles_deg + 360) % 360).int()  # Round to nearest integer
    return angles_int

def calculate_maae(pred_angles, true_angles):
    """Calculate Mean Absolute Angular Error"""
    if isinstance(pred_angles, torch.Tensor):
        pred_angles = pred_angles.detach().cpu().numpy()
    if isinstance(true_angles, torch.Tensor):
        true_angles = true_angles.detach().cpu().numpy()
    diff = np.abs(pred_angles - true_angles)
    angular_diff = np.minimum(diff, 360 - diff)
    return np.mean(angular_diff)

In [12]:
trainable_params = sum(p.numel() for p in model_eb2.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model_eb2.parameters())
print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%} of total)")

Trainable parameters: 6,285,948 (81.60% of total)


In [13]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 10)

    # Training phase
    model_eb2.train()
    train_loss = 0.0
    angle_preds, angle_truths = [], []
    
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images = images.to(device)
        angles = labels.to(device).float().view(-1, 1)  # Original angles
        
        # Convert angles to sin/cos targets
        sincos_targets = angle_to_sincos(angles)
        
        optimizer.zero_grad()
        sincos_outputs = model_eb2(images)  # Model predicts sin and cos directly
        
                # Calculate MSE loss between predicted and target sin/cos values
        loss = criterion(sincos_outputs, sincos_targets)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        
        # Convert sin/cos outputs back to angles for MAAE calculation
        predicted_angles = sincos_to_angle(sincos_outputs).view(-1, 1)
        
        angle_preds.append(predicted_angles.detach())
        angle_truths.append(angles.detach())

    train_total = len(train_loader.dataset)
    avg_train_loss = train_loss / train_total
    train_maae = calculate_maae(torch.cat(angle_preds), torch.cat(angle_truths))

    # Validation phase
    model_eb2.eval()
    val_loss = 0.0
    val_angle_preds, val_angle_truths = [], []

    with torch.no_grad():
        for images, labels in tqdm(validation_loader, desc="Validation", leave=False):
            images = images.to(device)
            angles = labels.to(device).float().view(-1, 1)
            
            # Convert angles to sin/cos targets
            sincos_targets = angle_to_sincos(angles)
            
            sincos_outputs = model_eb2(images)
            loss = criterion(sincos_outputs, sincos_targets)
            
            val_loss += loss.item() * images.size(0)
            
            # Convert sin/cos outputs back to angles for MAAE calculation
            predicted_angles = sincos_to_angle(sincos_outputs).view(-1, 1)
            
            val_angle_preds.append(predicted_angles)
            val_angle_truths.append(angles)

    val_total = len(validation_loader.dataset)
    avg_val_loss = val_loss / val_total
    val_maae = calculate_maae(torch.cat(val_angle_preds), torch.cat(val_angle_truths))

    # Update learning rate scheduler
    scheduler.step(avg_val_loss)

    # Print epoch results
    print(f"Train Loss: {avg_train_loss:.4f} | Train MAAE: {train_maae:.2f}")
    print(f"Val   Loss: {avg_val_loss:.4f} | Val   MAAE: {val_maae:.2f}")

    # Log metrics to WandB
    wandb.log({
        'avg_train_loss': avg_train_loss,
        'avg_val_loss': avg_val_loss,
        'train_maae': train_maae,
        'val_maae': val_maae,
        'epoch': epoch,
        'learning_rate': optimizer.param_groups[0]['lr']
    })

    # Check for improvement and save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        best_model_state = model_eb2.state_dict()
        print("✅ Validation loss improved. Saving model.")
    else:
        patience_counter += 1
        print(f"⚠️ No improvement in val loss. Patience {patience_counter}/{patience}")
        if patience_counter >= patience:
            print("⛔ Early stopping triggered.")
            break

# Save the best model
torch.save(best_model_state, 'best_model_sincos_eb2.pth')

# Log artifact to WandB
artifact = wandb.Artifact('efficientnet-sincos-model', type='model')
artifact.add_file('best_model_sincos_eb2.pth')
wandb.log_artifact(artifact)

Epoch 1/40
----------


Train Loss: 0.3098 | Train MAAE: 46.08
Val   Loss: 0.3264 | Val   MAAE: 46.10
✅ Validation loss improved. Saving model.
Epoch 2/40
----------


Train Loss: 0.1218 | Train MAAE: 21.15
Val   Loss: 0.2624 | Val   MAAE: 38.96
✅ Validation loss improved. Saving model.
Epoch 3/40
----------


Train Loss: 0.0783 | Train MAAE: 16.26
Val   Loss: 0.2392 | Val   MAAE: 36.21
✅ Validation loss improved. Saving model.
Epoch 4/40
----------


Train Loss: 0.0582 | Train MAAE: 13.66
Val   Loss: 0.2338 | Val   MAAE: 34.92
✅ Validation loss improved. Saving model.
Epoch 5/40
----------


...

Train Loss: 0.0046 | Train MAAE: 3.50
Val   Loss: 0.1707 | Val   MAAE: 26.51
⚠️ No improvement in val loss. Patience 2/5
Epoch 31/40
----------


Train Loss: 0.0045 | Train MAAE: 3.38
Val   Loss: 0.1726 | Val   MAAE: 27.02
⚠️ No improvement in val loss. Patience 3/5
Epoch 32/40
----------


Train Loss: 0.0046 | Train MAAE: 3.39
Val   Loss: 0.1713 | Val   MAAE: 26.94
⚠️ No improvement in val loss. Patience 4/5
Epoch 33/40
----------


Train Loss: 0.0042 | Train MAAE: 3.28
Val   Loss: 0.1728 | Val   MAAE: 27.56
⚠️ No improvement in val loss. Patience 5/5
⛔ Early stopping triggered.


<Artifact efficientnet-sincos-model>

In [16]:
# making the csv file
# validation transform 
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

labels_df = pd.read_csv(labels_val)
labels_df['filename'] = labels_df['filename'].apply(lambda x: os.path.join(images_val_path, x))
print(labels_df.head())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_eb2 = models.efficientnet_b2(pretrained = True)
in_features = model_eb2.classifier[1].in_features
model_eb2.classifier[1] = nn.Linear(in_features, 2)

model_eb2.load_state_dict(torch.load('best_model_sincos_eb2.pth'))
model_eb2 = model_eb2.to(device)
model_eb2.eval()

print("model loading done")

angles = []
true_angles = []

for idx, row in labels_df.iterrows():
    img = Image.open(row['filename']).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model_eb2(img_tensor)
        norm = torch.sqrt(output[:, 0]**2 + output[:, 1]**2).view(-1, 1)
        normalized_output = output / (norm + 1e-8)
        pred_angle = sincos_to_angle(normalized_output).item()

    angles.append(pred_angle)
    true_angles.append(row['angle'])

    if idx % 50 == 0:
        print(f"Processed {idx} / {len(labels_df)}")
def calculate_maae2(pred_angles, true_angles):
    pred_angles = np.array(pred_angles)
    true_angles = np.array(true_angles)
    diff = np.abs(pred_angles - true_angles)
    angular_diff = np.minimum(diff, 360 - diff)
    return np.mean(angular_diff)
# --- Compute final MAAE ---
maae_val = calculate_maae2(angles, true_angles)
print(f"📏 Validation MAAE: {maae_val:.2f}°")


val_df = pd.DataFrame({
    'id': labels_df.index,
    'angle': angles
})
val_df2 = pd.DataFrame({
    'id': labels_df.index,
    'angle': angles,
    'Real_Angle': labels_df['angle']
})

lenh = 369
test_df = pd.DataFrame({
    'id': list(range(369, 369 + lenh)),
    'angle': [0]*lenh
})
test_df2 = pd.DataFrame({
    'id': list(range(369, 369 + lenh)),
    'angle': [0]*lenh,
    'Real_Angle': [0]*lenh
})

# --- Combine and Save ---
submission_df = pd.concat([val_df, test_df], ignore_index=True)
submission_df2 = pd.concat([val_df2, test_df2], ignore_index=True)

submission_filename = '2022102070.csv'
submission_filename2 = '2022102070_withcorrect.csv'

submission_df.to_csv(submission_filename, index=False)
submission_df2.to_csv(submission_filename2, index=False)

print(f"✅ Submission saved as: {submission_filename}")
print(f"✅ Submission with real angles saved as: {submission_filename2}")

artifact = wandb.Artifact('prediction_results', type='dataset')
artifact.add_file(submission_filename)
wandb.log_artifact(artifact)

artifact = wandb.Artifact('prediction_results_with_real', type='dataset')
artifact.add_file(submission_filename2)
wandb.log_artifact(artifact)

wandb.log({"validation_maae": maae_val})

                                            filename timestamp  latitude  \
0  /kaggle/input/smai-s-25-section-a-project-phas...     15:04    219698   
1  /kaggle/input/smai-s-25-section-a-project-phas...     17:00    220182   
2  /kaggle/input/smai-s-25-section-a-project-phas...     17:00    220182   
3  /kaggle/input/smai-s-25-section-a-project-phas...     15:11    220195   
4  /kaggle/input/smai-s-25-section-a-project-phas...     17:35    220437   

   longitude  angle  Region_ID  
0     144782    311          2  
1     144211     89          2  
2     144211    177          2  
3     141942    301         12  
4     142673    323         12  


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B2_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_31/4286009153.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models 

model loading done
Processed 0 / 369
Processed 50 / 369
Processed 100 / 369
Processed 150 / 369
Processed 200 / 369
Processed 250 / 369
Processed 300 / 369
Processed 350 / 369
📏 Validation MAAE: 27.64°
✅ Submission saved as: 2022102070.csv
✅ Submission with real angles saved as: 2022102070_withcorrect.csv
wandb: 
wandb: 🚀 View run method2-efficientnetb2-angle-regression at: https://wandb.ai/v4ishnavi-iiit-hyderabad/smai-project-angleID/runs/ofo6lsw6
wandb: Find logs at: wandb/run-20250503_112416-ofo6lsw6/logs
